In [2]:
%matplotlib inline 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd 
from datetime import datetime
import tensorflow as tf 
from tqdm import tqdm
import pandas as pd 
import numpy as np 
import cv2 
import os 
import pickle 
import random 


# Image augmentation 
import albumentations as A

ModuleNotFoundError: No module named 'albumentations'

# 1. Data pre-processing